<a href="https://colab.research.google.com/github/mgerlach/advent_of_code/blob/main/2024/aoc2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Day 01, input

In [ ]:
input = [[int(i) for i in line.split()] for line in open('drive/MyDrive/AoC/2024/input01.txt')]

Day 01, part 1, sum of absolute diff of sorted list elements

In [ ]:
sum([abs(l - r) for (l, r) in zip(sorted([l for (l, _) in input]), sorted([r for (_, r) in input]))])

1530215

Day 01, part 2, sum lhs elements multiplied by frequency in rhs list

In [ ]:
from collections import Counter
r_counts = Counter([r for (_, r) in input])
sum([l * r_counts[l] for (l, _) in input])

26800609

Day 02, input

In [ ]:
input = [[int(i) for i in line.split()] for line in open('drive/MyDrive/AoC/2024/input02.txt')]

Day 02, part 1, part 1, determine (all increasing or all decreasing) and deltas < 4

In [ ]:
deltas = [[line[n+1] - line[n] for n in range(len(line)-1)] for line in input]

In [ ]:
def same_sgn(deltas_line):
  return all([deltas_line[i] * deltas_line[i+1] > 0 for i in range(len(deltas_line)-1)])

In [ ]:
def delta_max(deltas_line, m):
  return all([abs(d) <= m for d in deltas_line])

In [ ]:
sum([1 if same_sgn(deltas_line) and delta_max(deltas_line, 3) else 0 for deltas_line in deltas])

359

Day 02, part 2, allow removal of any single element

In [ ]:
def check_line(line):
  deltas = [line[n+1] - line[n] for n in range(len(line)-1)]
  return same_sgn(deltas) and delta_max(deltas, 3)

def remove1(line):
  return [line[:i] + line[i+1:] for i in range(len(line))]

# part 1 regression
sum([1 if check_line(line) else 0 for line in input])

# part 2
sum([1 if check_line(line) or any([check_line(r) for r in remove1(line)]) else 0 for line in input])

418

datetime.date(5, 12, 30)

Day 03, input

In [90]:
from functools import reduce
input = reduce(lambda s1, s2: s1 + s2, [line for line in open('drive/MyDrive/AoC/2024/input03.txt')], '')

Day 03, part 1, find mul(x,y) sequences, multiply and add

In [95]:
import re

def find_mul_and_eval(split):
  matches = re.findall("mul\((\d+),(\d+)\)", split)
  return sum([int(x) * int(y) for (x, y) in matches])

In [96]:
# input = "xmul(2,4)%&mul[3,7]!@^do_not_mul(5,5)+mul(32,64]then(mul(11,8)mul(8,5))"
find_mul_and_eval(input)

162813399

Day 03, part 2, evaluate do() and don't() sequences

In [97]:
import re

def next_state(state, split):
  (sum, is_active) = state
  match split:
    case "do()":
      return (sum, True)
    case "don't()":
      return (sum, False)
    case _:
      return (sum + (find_mul_and_eval(split) if is_active else 0), is_active)

# input = "xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5))"
reduce(next_state, re.split("(do\(\)|don't\(\))", input), (0, True))[0]

53783319